<a href="https://colab.research.google.com/github/wu-warren/BIGML_Application/blob/main/BigML_Research_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spuco

In [ ]:
# !pip install triton==3.2.0
# I ran into errors importing SpuCo package so installed an earlier version of triton

**1. Train a model using ERM**

In [ ]:
from spuco.datasets import SpuCoMNIST
from spuco.datasets.base_spuco_dataset import SpuriousFeatureDifficulty
import torchvision.transforms as T


# default parameters
trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=SpuriousFeatureDifficulty.MAGNITUDE_LARGE,
    spurious_correlation_strength=0.995,
        # high correlation strength since we are trying to test these methods on datasets with minority subclasses
    classes=[[0,1],[2,3],[4,5], [6,7],[8,9]], # superclasses
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=SpuriousFeatureDifficulty.MAGNITUDE_LARGE,
    classes=[[0,1],[2,3],[4,5], [6,7],[8,9]],
    split="test"
)
testset.initialize()


In [ ]:
T.ToPILImage()(trainset[19000][0]).resize((28,28))
# we see that the colored background is the spurious feature

In [ ]:
trainset[0][0]

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[0.3625, 0.3625, 0.3625,  ..., 0.3625, 0.3625, 0.3625],
         [0.3625, 0.3625, 0.3625,  ..., 0.3625, 0.3625, 0.3625],
         [0.3625, 0.3625, 0.3625,  ..., 0.3625, 0.3625, 0.

In [ ]:
from spuco.robust_train import ERM
import torch.nn as nn
import torch.nn.functional as F
from spuco.models import model_factory

# first tried with FFNN but switched to lenet after clustering did not yield clusters that captured the spurious features

# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(28 * 28 * 3, 256)  # input layer (flattened 28x28 pixel image with 3-color channel)
#         self.fc2 = nn.Linear(256, 128)      # hidden layer
#         self.fc3 = nn.Linear(128, 5)       # output layer (5 classes for the 5 superclasses)

#     def forward(self, x):
#         x = x.view(-1, 28 * 28 * 3)  # flatten the image into 28 * 28 * 3 element vector
#         x = F.relu(self.fc1(x)) # relu for introducing nonlinearity
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x # no softmax since we are going to use this output for clustering

# model = Net()

In [ ]:
import torch.optim as optim

model = model_factory('lenet', trainset[0][0].shape, trainset.num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer

erm = ERM(
    model = model,
    trainset = trainset,
    batch_size = 64,
    optimizer = optimizer,
    num_epochs = 1,
    device = "cpu",
    verbose = True
)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
erm.train()

Epoch 0:   0%|          | 0/751 [00:00<?, ?batch/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 0:   1%|          | 6/751 [00:00<00:39, 18.70batch/s, accuracy=64.0625%, loss=1.46]

ERM | Epoch 0 | Loss: 1.617039442062378 | Accuracy: 25.0%
ERM | Epoch 0 | Loss: 1.6198195219039917 | Accuracy: 12.5%
ERM | Epoch 0 | Loss: 1.6000875234603882 | Accuracy: 20.3125%
ERM | Epoch 0 | Loss: 1.5937166213989258 | Accuracy: 17.1875%
ERM | Epoch 0 | Loss: 1.5729663372039795 | Accuracy: 28.125%
ERM | Epoch 0 | Loss: 1.579785943031311 | Accuracy: 14.0625%
ERM | Epoch 0 | Loss: 1.5661741495132446 | Accuracy: 20.3125%
ERM | Epoch 0 | Loss: 1.55601167678833 | Accuracy: 28.125%
ERM | Epoch 0 | Loss: 1.508431315422058 | Accuracy: 48.4375%
ERM | Epoch 0 | Loss: 1.4616222381591797 | Accuracy: 64.0625%


Epoch 0:   2%|▏         | 15/751 [00:00<00:23, 30.82batch/s, accuracy=68.75%, loss=0.784]

ERM | Epoch 0 | Loss: 1.4255963563919067 | Accuracy: 59.375%
ERM | Epoch 0 | Loss: 1.3774404525756836 | Accuracy: 57.8125%
ERM | Epoch 0 | Loss: 1.2933591604232788 | Accuracy: 59.375%
ERM | Epoch 0 | Loss: 1.2227082252502441 | Accuracy: 64.0625%
ERM | Epoch 0 | Loss: 1.1701159477233887 | Accuracy: 57.8125%
ERM | Epoch 0 | Loss: 1.1078497171401978 | Accuracy: 53.125%
ERM | Epoch 0 | Loss: 0.9932467937469482 | Accuracy: 45.3125%
ERM | Epoch 0 | Loss: 0.9066041111946106 | Accuracy: 43.75%
ERM | Epoch 0 | Loss: 0.7841891646385193 | Accuracy: 68.75%


Epoch 0:   3%|▎         | 25/751 [00:00<00:18, 38.26batch/s, accuracy=98.4375%, loss=0.201]

...

Epoch 0:  99%|█████████▉| 743/751 [00:19<00:00, 38.19batch/s, accuracy=96.875%, loss=0.256]

ERM | Epoch 0 | Loss: 0.007629780564457178 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.010158208198845387 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.010434936732053757 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.00705661391839385 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.1476878821849823 | Accuracy: 98.4375%
ERM | Epoch 0 | Loss: 0.006909712217748165 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.005298929288983345 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.004101310856640339 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.2563197910785675 | Accuracy: 96.875%


Epoch 0: 100%|█████████▉| 748/751 [00:20<00:00, 41.07batch/s, accuracy=100.0%, loss=0.00474]

ERM | Epoch 0 | Loss: 0.12349579483270645 | Accuracy: 98.4375%
ERM | Epoch 0 | Loss: 0.004005237948149443 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.0037333930376917124 | Accuracy: 100.0%
ERM | Epoch 0 | Loss: 0.11890167742967606 | Accuracy: 98.4375%
ERM | Epoch 0 | Loss: 0.0047407145611941814 | Accuracy: 100.0%


Epoch 0: 100%|██████████| 751/751 [00:20<00:00, 37.16batch/s, accuracy=100.0%, loss=0.00474]


In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=testset.group_weights,
    batch_size=64,
    model=model,
    device="cpu",
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:11,  2.14it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:11,  2.08it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:10,  2.12it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:12<00:00,  1.52it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:13<00:00,  1.89it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [ ]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [ ]:
evaluator.average_accuracy

20.0

In [ ]:
evaluator.evaluate_spurious_attribute_prediction()

20.29

In [ ]:
print(trainset.group_weights)
print(testset.group_weights)

# poor evaluations on test set cases where label =/= spurious label
# this makes sense when you see that there are very few samples in the training
# set where this is the case

{(2, 2): 0.1867552703941338, (0, 0): 0.2100241646529456, (4, 4): 0.19567119406716107, (1, 1): 0.20046246146154487, (3, 3): 0.20202483126406132, (4, 1): 0.0002916423631364053, (1, 4): 0.0002916423631364053, (3, 4): 0.0002708107657695192, (2, 1): 0.0002499791684026331, (3, 1): 0.0002499791684026331, (1, 0): 0.0003541371552370636, (1, 2): 0.00018748437630197485, (1, 3): 0.00018748437630197485, (2, 3): 0.00018748437630197485, (4, 0): 0.00016665277893508876, (0, 1): 0.00016665277893508876, (4, 3): 0.00016665277893508876, (4, 2): 0.0003749687526039497, (0, 3): 0.0003124739605032914, (0, 2): 0.0004166319473377219, (0, 4): 0.00016665277893508876, (3, 2): 0.00020831597366886094, (2, 0): 0.00018748437630197485, (2, 4): 0.0003333055578701775, (3, 0): 0.0002916423631364053}
{(3, 3): 0.0397, (1, 0): 0.0409, (0, 4): 0.0423, (0, 0): 0.0423, (2, 3): 0.0375, (0, 2): 0.0423, (2, 0): 0.0375, (4, 4): 0.0396, (2, 4): 0.0374, (4, 0): 0.0397, (4, 1): 0.0397, (0, 1): 0.0423, (2, 1): 0.0375, (3, 1): 0.0397, (1

**2. Clustering based on output produced for ERM**

In [ ]:
from spuco.utils import get_model_outputs

erm_superclass_pred = get_model_outputs(
    model = model,
    dataset = trainset,
    device = "cpu",
    verbose = True
)

Getting model outputs: 100%|██████████| 751/751 [00:09<00:00, 80.60batch/s]


In [ ]:
erm_superclass_pred[0:5]

tensor([[-0.7980, -1.0649,  7.8168,  0.1162, -1.2008],
        [ 5.2946, -1.2267, -0.8801, -0.5173, -0.0895],
        [-0.8492, -1.0603,  8.3519,  0.0234, -1.2803],
        [ 5.4859, -1.3604, -0.8865, -0.5351, -0.1236],
        [-1.8198, -1.1841, -0.5390,  0.4735,  6.1069]])

In [ ]:
from spuco.group_inference import Cluster

cluster = Cluster(
    # clustering within each superclass
    Z = erm_superclass_pred,
    class_labels = trainset.labels,
    max_clusters = 5,
    verbose = True
)

cluster_partition = (cluster.infer_groups())

Clustering class-wise:   0%|          | 0/5 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.9641191
For n_clusters = 3 The average silhouette_score is : 0.96420586
For n_clusters = 4 The average silhouette_score is : 0.52052397


Clustering class-wise:  20%|██        | 1/5 [00:05<00:23,  5.90s/it]

For n_clusters = 5 The average silhouette_score is : 0.52145165
For n_clusters = 2 The average silhouette_score is : 0.95720077
For n_clusters = 3 The average silhouette_score is : 0.9562984
For n_clusters = 4 The average silhouette_score is : 0.60623664


Clustering class-wise:  40%|████      | 2/5 [00:14<00:21,  7.26s/it]

...

For n_clusters = 5 The average silhouette_score is : 0.9686932
For n_clusters = 2 The average silhouette_score is : 0.967947
For n_clusters = 3 The average silhouette_score is : 0.9683742
For n_clusters = 4 The average silhouette_score is : 0.96947134


Clustering class-wise:  80%|████████  | 4/5 [00:33<00:09,  9.32s/it]

For n_clusters = 5 The average silhouette_score is : 0.9704802
For n_clusters = 2 The average silhouette_score is : 0.971122
For n_clusters = 3 The average silhouette_score is : 0.97154003
For n_clusters = 4 The average silhouette_score is : 0.9706469


Clustering class-wise: 100%|██████████| 5/5 [00:45<00:00,  9.09s/it]

For n_clusters = 5 The average silhouette_score is : 0.97181666


In [ ]:
for key in sorted(cluster_partition.keys()):
  print(f"{key}: {len(cluster_partition[key])}")

(0, 0): 10082
(0, 1): 51
(1, 0): 9623
(1, 1): 17
(1, 2): 14
(1, 3): 9
(1, 4): 9
(2, 0): 8965
(2, 1): 34
(2, 2): 12
(3, 0): 9698
(3, 1): 12
(3, 2): 10
(3, 3): 14
(3, 4): 13
(4, 0): 9409
(4, 1): 14
(4, 2): 18


In [ ]:
for key, indices in sorted(cluster_partition.items()):
    print(f"Displaying image from cluster {key}:")
    display(T.ToPILImage()(trainset[indices[0]][0]).resize((28,28)))

Displaying image from cluster (0, 0):


Displaying image from cluster (0, 1):


Displaying image from cluster (1, 0):


Displaying image from cluster (1, 1):


Displaying image from cluster (1, 2):


...

Displaying image from cluster (3, 3):


Displaying image from cluster (3, 4):


Displaying image from cluster (4, 0):


Displaying image from cluster (4, 1):


Displaying image from cluster (4, 2):


**3. Group Balancing**

In [ ]:
from spuco.robust_train import GroupBalanceBatchERM

# same model arch from part 1

gbb_model = model_factory('lenet', trainset[0][0].shape, trainset.num_classes)

gbb_erm = GroupBalanceBatchERM(
    model = gbb_model,
    trainset = trainset,
    group_partition = cluster_partition,
    batch_size = 64,
    optimizer = optimizer,
    num_epochs = 1,
    device = "cpu",
    verbose = True
)

gbb_erm.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   0%|          | 0/751 [00:00<?, ?batch/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 0:   1%|          | 6/751 [00:00<00:38, 19.34batch/s, accuracy=20.3125%, loss=1.61]

GB | Epoch 0 | Loss: 1.6061931848526 | Accuracy: 21.875%
GB | Epoch 0 | Loss: 1.6061731576919556 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6130945682525635 | Accuracy: 12.5%
GB | Epoch 0 | Loss: 1.6098393201828003 | Accuracy: 23.4375%
GB | Epoch 0 | Loss: 1.6102557182312012 | Accuracy: 9.375%
GB | Epoch 0 | Loss: 1.6126949787139893 | Accuracy: 10.9375%
GB | Epoch 0 | Loss: 1.5978373289108276 | Accuracy: 28.125%
GB | Epoch 0 | Loss: 1.6077436208724976 | Accuracy: 20.3125%
GB | Epoch 0 | Loss: 1.6180031299591064 | Accuracy: 12.5%
GB | Epoch 0 | Loss: 1.6063393354415894 | Accuracy: 20.3125%


Epoch 0:   2%|▏         | 16/751 [00:00<00:21, 33.94batch/s, accuracy=25.0%, loss=1.6]    

GB | Epoch 0 | Loss: 1.6156811714172363 | Accuracy: 10.9375%
GB | Epoch 0 | Loss: 1.6078948974609375 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.6084556579589844 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6106401681900024 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6120940446853638 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.6143401861190796 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.61180579662323 | Accuracy: 23.4375%
GB | Epoch 0 | Loss: 1.6075032949447632 | Accuracy: 10.9375%
GB | Epoch 0 | Loss: 1.6020143032073975 | Accuracy: 25.0%


Epoch 0:   3%|▎         | 26/751 [00:00<00:18, 39.02batch/s, accuracy=25.0%, loss=1.61]

...

Epoch 0: 100%|█████████▉| 748/751 [00:31<00:00, 24.44batch/s, accuracy=20.3125%, loss=1.61]

GB | Epoch 0 | Loss: 1.6136554479599 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6085399389266968 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.609065294265747 | Accuracy: 17.1875%
GB | Epoch 0 | Loss: 1.6035170555114746 | Accuracy: 18.75%
GB | Epoch 0 | Loss: 1.6081349849700928 | Accuracy: 20.3125%
GB | Epoch 0 | Loss: 1.6069419384002686 | Accuracy: 20.3125%


Epoch 0: 100%|██████████| 751/751 [00:31<00:00, 25.53batch/s, accuracy=50.0%, loss=1.58]

GB | Epoch 0 | Loss: 1.608494520187378 | Accuracy: 14.0625%
GB | Epoch 0 | Loss: 1.582450032234192 | Accuracy: 50.0%


Epoch 0: 100%|██████████| 751/751 [00:31<00:00, 23.67batch/s, accuracy=50.0%, loss=1.58]


In [ ]:
gbb_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=gbb_model,
    device="cpu",
    verbose=True
)
gbb_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:20,  1.19it/s]

Group (0, 0) Accuracy: 0.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:19,  1.18it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:02<00:17,  1.22it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:23<00:01,  1.08s/it]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:24<00:00,  1.04it/s]

Group (4, 4) Accuracy: 0.0


{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (1, 2): 0.0,
 (1, 3): 10.049019607843137,
 (1, 4): 0.0,
 (2, 0): 100.0,
 (2, 1): 100.0,
 (2, 2): 100.0,
 (2, 3): 99.2,
 (2, 4): 100.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 0.0}

In [ ]:
gbb_evaluator.worst_group_accuracy

((0, 0), 0.0)

In [ ]:
gbb_evaluator.average_accuracy

18.773086433973642

In [ ]:
gbb_evaluator.evaluate_spurious_attribute_prediction()

19.95

Conclusion:

- The retrained model appears to predict superclass almost exclusively
- Possible next steps: reduce overfitting (regularization, reduce model complexity, etc.)